In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data1 = np.genfromtxt('/content/drive/My Drive/Data ML/ec014.42_794_796_798.txt', usecols=list(range(0,256)), skip_header=1)
data2 = np.genfromtxt('/content/drive/My Drive/Data ML/ec014.42_795_797_spikes.txt', usecols=list(range(0,256)), skip_header=1)
parameters1 = np.genfromtxt('/content/drive/My Drive/Data ML/ec014.42_794_796_798.txt', dtype=None, encoding='UTF-8', usecols=list(range(256,267)), skip_header=1)
parameters2 = np.genfromtxt('/content/drive/My Drive/Data ML/ec014.42_795_797_spikes.txt', dtype=None, encoding='UTF-8', usecols=list(range(256,267)), skip_header=1)

2.) Classification of excitatory/inhibitory (over all electrodes in 3 session 794, 796, 798)


In [ ]:
#Classication in 2 groups (excitatory/inhibitory)
from keras.constraints import maxnorm

#Adjust data
data = np.genfromtxt('/content/drive/My Drive/ec014.42_794_796_798.txt', usecols=list(range(0,256)))
parameters = np.genfromtxt('/content/drive/My Drive/ec014.42_794_796_798.txt', dtype=None, encoding='UTF-8', usecols=list(range(256,267)))

print(data.shape)

classification = np.empty(shape=(data.shape[0],1))
for i in range(0, data.shape[0]):
  if parameters[i][8] != '0' and parameters[i][9] == '0': #excit
    classification[i] = 0
  elif parameters[i][8] == '0' and parameters[i][9] != '0': #inhib
    classification[i] = 1
  elif parameters[i][8] != '0' and parameters[i][9] != '0': #both-con
    classification[i] = 3
  else:
    classification[i] = 2 #neither
  i=i+1
data = np.append(data, classification, axis=1)
data =data[data[:,256] != 3] #No double con


#Divide and reshape data randomly in subset for training and validation
training_labels = data[0:(data.shape[0]-10000),256]
training_data = data[0:(data.shape[0]-10000),0:256]
test_labels = data[(data.shape[0]-10000):(data.shape[0]),256]
test_data = data[(data.shape[0]-10000):(data.shape[0]),0:256]
training_data = training_data.reshape(training_data.shape[0], training_data.shape[1],1)
test_data = test_data.reshape(test_data.shape[0], test_data.shape[1],1)

print("Excitatory:", len(classification[classification[:,0] == 0]))
print("Inhibitory:", len(classification[classification[:,0] == 1]))
print("Neither:", len(classification[classification[:,0] == 2]))
print("total:", data.shape[0])

model = tf.keras.models.Sequential([tf.keras.layers.Conv1D(filters=256, kernel_size=16, activation='relu', input_shape=(256,1)),
                                    tf.keras.layers.Conv1D(filters=256, kernel_size=16, activation='relu'),
                                    tf.keras.layers.MaxPooling1D(pool_size=3),
                                    tf.keras.layers.Conv1D(filters=128, kernel_size=8, activation='relu'),
                                    tf.keras.layers.Conv1D(filters=128, kernel_size=8, activation='relu'),
                                    tf.keras.layers.MaxPooling1D(pool_size=3),
                                    tf.keras.layers.Conv1D(filters=100, kernel_size=4, activation='relu'),
                                    tf.keras.layers.Conv1D(filters=100, kernel_size=4, activation='relu'),
                                    tf.keras.layers.MaxPooling1D(pool_size=3),
                                    tf.keras.layers.Conv1D(filters=100, kernel_size=2, activation='relu'),
                                    tf.keras.layers.Conv1D(filters=100, kernel_size=2, activation='relu'),
                                    tf.keras.layers.GlobalAveragePooling1D(), 
                                    tf.keras.layers.Dropout(0.3),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512, activation='relu', kernel_constraint=maxnorm(3)),
                                    tf.keras.layers.Dense(3, activation=tf.nn.softmax)])
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_data, training_labels, epochs=5)

print("The model classifies accurately in", model.evaluate(test_data, test_labels)[1]*100, "%")

(81931, 256)
Excitatory: 13155
Inhibitory: 38497
Neither: 30278
total: 81930
Epoch 1/5
2248/2248 [==============================] - 2450s 1s/step - loss: 0.1831 - accuracy: 0.9331
Epoch 2/5
2248/2248 [==============================] - 2453s 1s/step - loss: 0.1024 - accuracy: 0.9652
Epoch 3/5
2248/2248 [==============================] - 2438s 1s/step - loss: 0.0882 - accuracy: 0.9713
Epoch 4/5
2248/2248 [==============================] - 2429s 1s/step - loss: 0.0791 - accuracy: 0.9735
Epoch 5/5
313/313 [==============================] - 85s 272ms/step - loss: 0.1079 - accuracy: 0.9708
The model classifies accurately in 97.079998254776 %
